Dining Philosophers Problem
Using Thread and Semaphores

In [ ]:
import threading
import time
import random
import logging
from threading import Semaphore

# Logger setup for file output
logging.basicConfig(filename='dining_philosophers_improved.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Number of philosophers
NUM_PHILOSOPHERS = 5

# Chopsticks and waiter semaphore
chopsticks = [Semaphore(1) for _ in range(NUM_PHILOSOPHERS)]
waiter = Semaphore(NUM_PHILOSOPHERS - 1)  # Allow at most N-1 philosophers to access chopsticks simultaneously

# Philosopher class
class Philosopher(threading.Thread):
    def __init__(self, philosopher_number, meals_limit):
        threading.Thread.__init__(self)
        self.philosopher_number = philosopher_number
        self.meals_eaten = 0
        self.meals_limit = meals_limit  # Philosopher will stop after eating this many times

    def run(self):
        while self.meals_eaten < self.meals_limit:
            self.think()
            self.pick_up_chopsticks()
            self.eat()
            self.put_down_chopsticks()

    def think(self):
        logging.info(f"Philosopher {self.philosopher_number} is thinking.")
        time.sleep(random.uniform(1, 3))  # Simulate thinking time

    def pick_up_chopsticks(self):
        logging.info(f"Philosopher {self.philosopher_number} is hungry and waiting for chopsticks.")
        waiter.acquire()  # Ensures that at most N-1 philosophers can attempt to pick up chopsticks

        # Pick up the lower-numbered chopstick first (avoids deadlock)
        left_chopstick = self.philosopher_number
        right_chopstick = (self.philosopher_number + 1) % NUM_PHILOSOPHERS

        chopsticks[left_chopstick].acquire()
        logging.info(f"Philosopher {self.philosopher_number} picked up left chopstick {left_chopstick}.")

        chopsticks[right_chopstick].acquire()
        logging.info(f"Philosopher {self.philosopher_number} picked up right chopstick {right_chopstick}.")

    def eat(self):
        logging.info(f"Philosopher {self.philosopher_number} is eating. Meal number: {self.meals_eaten + 1}")
        time.sleep(random.uniform(1, 2))  # Simulate eating time
        self.meals_eaten += 1

    def put_down_chopsticks(self):
        left_chopstick = self.philosopher_number
        right_chopstick = (self.philosopher_number + 1) % NUM_PHILOSOPHERS

        chopsticks[left_chopstick].release()
        logging.info(f"Philosopher {self.philosopher_number} put down left chopstick {left_chopstick}.")

        chopsticks[right_chopstick].release()
        logging.info(f"Philosopher {self.philosopher_number} put down right chopstick {right_chopstick}.")

        waiter.release()  # Allow another philosopher to attempt eating

def run_philosophers(meals_limit):
    # Create philosopher threads with a meals limit
    philosophers = [Philosopher(i, meals_limit) for i in range(NUM_PHILOSOPHERS)]

    # Start all philosopher threads
    for philosopher in philosophers:
        philosopher.start()

    # Join all philosopher threads to ensure they complete
    for philosopher in philosophers:
        philosopher.join()

    logging.info("All philosophers have finished their meals.")

if __name__ == "__main__":
    # Define how many meals each philosopher is allowed to eat (for example, 3)
    meals_limit = 3
    run_philosophers(meals_limit)